In [1]:
!pip install --upgrade transformers soundfile mistral-common

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 34.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.54.0
    Uninstalling transformers-4.54.0:
      Successfully uninstalled transformers-4.54.0


In [2]:
from huggingface_hub import login
login("hf_")

In [3]:
from transformers import VoxtralProcessor, VoxtralForConditionalGeneration
import torch
import torchaudio

In [4]:
# Load model and correct processor
repo = "mistralai/Voxtral-Mini-3B-2507"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
processor = VoxtralProcessor.from_pretrained(repo)
model = VoxtralForConditionalGeneration.from_pretrained(
    repo, torch_dtype=torch.bfloat16, device_map=device
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

tekken.json:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.38G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/108 [00:00<?, ?B/s]

In [6]:
processor

VoxtralProcessor:
- feature_extractor: WhisperFeatureExtractor {
  "chunk_length": 30,
  "dither": 0.0,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 128,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "VoxtralProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: <transformers.tokenization_mistral_common.MistralCommonTokenizer object at 0x7b71faf78b10>

{
  "processor_class": "VoxtralProcessor"
}

In [14]:
# Load and preprocess audio
audio_path = "001_t2.wav"
waveform, sr = torchaudio.load(audio_path)

# Resample if not 16kHz
if sr != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
    waveform = resampler(waveform)
    sr = 16000

# Ensure mono audio
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

# Prepare input - provide required text parameter
inputs = processor(
    text="",  # Empty string for transcription task
    audio=waveform.squeeze().numpy(),
    sampling_rate=sr,
    return_tensors="pt"
)

# Move to model device
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Run inference
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=256)

# Decode
transcription = processor.batch_decode(outputs, skip_special_tokens=True)[0]
print("Transcription:", transcription)

ValueError: Kwargs ['return_dict', 'tokenize'] are not supported by `MistralCommonTokenizer.__call__`.

In [16]:
conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "audio",
                "path": "https://huggingface.co/datasets/hf-internal-testing/dummy-audio-samples/resolve/main/winning_call.mp3",
            },
        ],
    }
]

inputs = processor.apply_chat_template(conversation)
inputs = inputs.to(device, dtype=torch.bfloat16)

outputs = model.generate(**inputs, max_new_tokens=500)
decoded_outputs = processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)

print("\nGenerated response:")
print("=" * 80)
print(decoded_outputs[0])
print("=" * 80)



Generated response:
The audio describes a dramatic moment in a baseball game, likely a playoff or championship game, where a home run is hit, and the team is celebrating their potential to win the American League Championship. Here's a breakdown:

- **Pitcher**: The pitcher is identified as 0-1, meaning he has not yet thrown a strike in the at-bat.
- **Batter**: The batter is Edgar Martinez, a known player in the MLB.
- **Hit**: The pitch is swung on and hit, resulting in a home run.
- **Outfield**: The ball is hit to left field, and the outfielder is trying to catch it.
- **Base Running**: The runner, likely named Joy, is at third base. The team is trying to wave him in, indicating a potential run-scoring play.
- **Plate**: The throw to the plate is late, meaning the runner at third base has a chance to score.
- **Celebration**: The team is excited and celebrating, with the announcer expressing disbelief and amazement at the ongoing success.

The audio suggests that the team is in a 

In [18]:
inputs = processor.apply_transcription_request(language="en", audio="https://huggingface.co/datasets/hf-internal-testing/dummy-audio-samples/resolve/main/obama.mp3", model_id=repo)
inputs = inputs.to(device, dtype=torch.bfloat16)

outputs = model.generate(**inputs, max_new_tokens=500)
decoded_outputs = processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)

print("\nGenerated responses:")
print("=" * 80)
for decoded_output in decoded_outputs:
    print(decoded_output)
    print("=" * 80)



Generated responses:
This week, I traveled to Chicago to deliver my final farewell address to the nation, following in the tradition of presidents before me. It was an opportunity to say thank you. Whether we've seen eye-to-eye or rarely agreed at all, my conversations with you, the American people, in living rooms and schools, at farms and on factory floors, at diners and on distant military outposts, All these conversations are what have kept me honest, kept me inspired, and kept me going. Every day, I learned from you. You made me a better president, and you made me a better man. Over the course of these eight years, I've seen the goodness, the resilience, and the hope of the American people. I've seen neighbors looking out for each other as we rescued our economy from the worst crisis of our lifetimes. I've hugged cancer survivors who finally know the security of affordable health care. I've seen communities like Joplin rebuild from disaster, and cities like Boston show the world 

In [19]:
inputs = processor.apply_transcription_request(language="en", audio="/content/001_t2.mp3", model_id=repo)
inputs = inputs.to(device, dtype=torch.bfloat16)

outputs = model.generate(**inputs, max_new_tokens=500)
decoded_outputs = processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)

print("\nGenerated responses:")
print("=" * 80)
for decoded_output in decoded_outputs:
    print(decoded_output)
    print("=" * 80)



Generated responses:
Thank you. Your name is from Max Life Insurance. We are calling from the service department. Can we talk about the policy for a minute? Are you free? Thank you. You have issued a policy from Max Life Fast Right Super Plan. Policy number 600603260. The premium rate is 35th of the 90th month of 2021. The premium amount is 1,20,000. The policy status is discontinued. Why are you not paying? I don't know. Sorry, sir. Finance, sir. 1,20,000, now you can take it. Yes, sir. Can we take the point from the finance? How long can we take it? So, the locking period is 5 years. You have taken the policy in 2018. You have paid for 18, 19, 20, 21, 3 years. You can take the amount after paying the premium. The locking period is complete. You can take it. How much will it take? Sir, if you have enough funds, you can take the amount. Otherwise, you can do a partial withdrawal. Sir, as of now, there is 1,87,551 rupees. But if you make the payment now, your funds will participate in 

In [21]:
"Thank you. Your name is from Max Life Insurance. We are calling from the service department. Can we talk about the policy for a minute? Are you free? Thank you. You have issued a policy from Max Life Fast Right Super Plan. Policy number 600603260. The premium rate is 35th of the 90th month of 2021. The premium amount is 1,20,000. The policy status is discontinued. Why are you not paying? I don't know. Sorry, sir. Finance, sir. 1,20,000, now you can take it. Yes, sir. Can we take the point from the finance? How long can we take it? So, the locking period is 5 years. You have taken the policy in 2018. You have paid for 18, 19, 20, 21, 3 years. You can take the amount after paying the premium. The locking period is complete. You can take it. How much will it take? Sir, if you have enough funds, you can take the amount. Otherwise, you can do a partial withdrawal. Sir, as of now, there is 1,87,551 rupees. But if you make the payment now, your funds will participate in the share market. So, high returns will also come with chances. Sir, your policy status is discontinued. Your funds are only in the discontinued fund. The policy status is active only. Your funds will participate only if the policy status is active. So, you have not paid for 2021. So, how will your funds participate? He has taken a loan of 1,20,000. When will the policy end? 5 years. 5 years you have paid. But the policy will end only after 10 years. After 10 years, your funds will participate. Then you can do a partial withdrawal. Then you can take the amount. You can take it. So, the locking period is complete. When will the locking period be complete? When will it open? When will the policy start? You have taken it in 2018. Which month? 9th month, 35th day you have taken it. Can we take it in 39"

"Thank you. Your name is from Max Life Insurance. We are calling from the service department. Can we talk about the policy for a minute? Are you free? Thank you. You have issued a policy from Max Life Fast Right Super Plan. Policy number 600603260. The premium rate is 35th of the 90th month of 2021. The premium amount is 1,20,000. The policy status is discontinued. Why are you not paying? I don't know. Sorry, sir. Finance, sir. 1,20,000, now you can take it. Yes, sir. Can we take the point from the finance? How long can we take it? So, the locking period is 5 years. You have taken the policy in 2018. You have paid for 18, 19, 20, 21, 3 years. You can take the amount after paying the premium. The locking period is complete. You can take it. How much will it take? Sir, if you have enough funds, you can take the amount. Otherwise, you can do a partial withdrawal. Sir, as of now, there is 1,87,551 rupees. But if you make the payment now, your funds will participate in the share market. So,